In [27]:
from textblob import TextBlob
import numpy as np
import pandas as pd
import csv
import pickle

In [17]:
train_df = pd.read_csv('train_new.csv', encoding='latin-1')

In [22]:
test_df = pd.read_csv('test.csv', encoding='latin-1')

In [23]:
attributes_df = pd.read_csv('attributes.csv', encoding='latin-1')

In [24]:
descriptions_df = pd.read_csv('product_descriptions.csv', encoding='latin-1')

In [25]:
def extract(text):
    return TextBlob(text).tokens.lower()

In [28]:
train_df['title'] = train_df.product_title.map(extract)

In [31]:
train_df = train_df.set_index('product_uid') 

In [33]:
descriptions_df = descriptions_df.set_index('product_uid')

In [34]:
joined = train_df.join(descriptions_df)

In [35]:
joined.head()

,id,product_title,search_term,relevance,title,product_description
product_uid,,,,,,
100001,2,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"[simpson, strong-tie, 12-gauge, angle]","Not only do angles make joints stronger, they ..."
100001,3,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"[simpson, strong-tie, 12-gauge, angle]","Not only do angles make joints stronger, they ..."
100002,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,"[behr, premium, textured, deckover, 1-gal, ., ...",BEHR Premium Textured DECKOVER is an innovativ...
100005,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,"[delta, vero, 1-handle, shower, only, faucet, ...",Update your bathroom with the Delta Vero Singl...
100005,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,"[delta, vero, 1-handle, shower, only, faucet, ...",Update your bathroom with the Delta Vero Singl...


In [36]:
joined = joined[['id', 'title', 'product_description', 'search_term', 'relevance']]
joined.columns = ['search_id', 'title', 'desc', 'search_term', 'relevance']

In [38]:
joined.head()

,search_id,title,desc,search_term,relevance
product_uid,,,,,
100001,2,"[simpson, strong-tie, 12-gauge, angle]","Not only do angles make joints stronger, they ...",angle bracket,3.00
100001,3,"[simpson, strong-tie, 12-gauge, angle]","Not only do angles make joints stronger, they ...",l bracket,2.50
100002,9,"[behr, premium, textured, deckover, 1-gal, ., ...",BEHR Premium Textured DECKOVER is an innovativ...,deck over,3.00
100005,16,"[delta, vero, 1-handle, shower, only, faucet, ...",Update your bathroom with the Delta Vero Singl...,rain shower head,2.33
100005,17,"[delta, vero, 1-handle, shower, only, faucet, ...",Update your bathroom with the Delta Vero Singl...,shower only faucet,2.67


In [39]:
joined.desc = joined.desc.map(extract)

In [40]:
joined.search_term = joined.search_term.map(extract)

In [42]:
joined['title_desc'] = joined.title + joined.desc

In [44]:
def compare(row):
    search, desc = row.search_term, row.title_desc
    return len(set(search) & (set(desc))) / len(set(search))

In [ ]:
joined['score'] = joined.applymap(compare)

In [54]:
joined['a'] = joined.title_desc.map(lambda x: set(x))
joined['b'] = joined.search_term.map(lambda x: set(x))

ValueError: Length of values does not match length of index

In [67]:
score_df = pd.DataFrame([{'id':id_, 'score': len(row.a & row.b) / len(row.b)} for id_, row in joined.iterrows()]).set_index('id')

In [69]:
joined = joined.join(score_df)

In [72]:
joined.head()

,search_id,title,desc,search_term,relevance,title_desc,a,b,score
100001,2,"[simpson, strong-tie, 12-gauge, angle]","[not, only, do, angles, make, joints, stronger...","[angle, bracket]",3.0,"[simpson, strong-tie, 12-gauge, angle, not, on...","set([projectsstronger, end, common, consistent...","set([angle, bracket])",0.5
100001,2,"[simpson, strong-tie, 12-gauge, angle]","[not, only, do, angles, make, joints, stronger...","[angle, bracket]",3.0,"[simpson, strong-tie, 12-gauge, angle, not, on...","set([projectsstronger, end, common, consistent...","set([angle, bracket])",0.0
100001,3,"[simpson, strong-tie, 12-gauge, angle]","[not, only, do, angles, make, joints, stronger...","[l, bracket]",2.5,"[simpson, strong-tie, 12-gauge, angle, not, on...","set([projectsstronger, end, common, consistent...","set([l, bracket])",0.5
100001,3,"[simpson, strong-tie, 12-gauge, angle]","[not, only, do, angles, make, joints, stronger...","[l, bracket]",2.5,"[simpson, strong-tie, 12-gauge, angle, not, on...","set([projectsstronger, end, common, consistent...","set([l, bracket])",0.0
100002,9,"[behr, premium, textured, deckover, 1-gal, ., ...","[behr, premium, textured, deckover, is, an, in...","[deck, over]",3.0,"[behr, premium, textured, deckover, 1-gal, ., ...","set([1-gal, products, the, premium, resin, rej...","set([over, deck])",0.5


In [76]:
joined.corr()

,search_id,relevance,score
search_id,1.000000,-0.057793,0.080292
relevance,-0.057793,1.000000,0.194872
score,0.080292,0.194872,1.000000


In [80]:
joined[['search_id', 'title', 'desc', 'title_desc', 'search_term', 'relevance', 'score']].to_csv('./joined.csv')